In [1]:
#import julia; julia.install(quiet=True)
from julia import Main

import numpy     as np
import panel     as pn; pn.extension()
import holoviews as hv; hv.extension( "bokeh", logo=False)

from panel.interact import interact

def raster(img):  return hv.Image(img).opts(cmap="gray", xaxis=None, yaxis=None, frame_width=200, aspect='equal')

In [2]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [3]:
%%julia
using LinearAlgebra;

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Generalized Minimum Residual (GMRES)</strong>
</div></div>

# 1. Solving $\mathbf{A x = b}$ as an Optimization Problem

**Problem:** The solution algorithms for $A x = b$ based on GE, QR or the SVD<br>
$\qquad$ require $\mathcal{O}(n^3)$ operations, where $A$ is size $n \times n$.

For large matrices, we need better: Iterative schemes can be designed for different type of matrices.<br>
$\qquad$ the following algoritm tends to work well for large sparse matrices.

## 1.1 Idea

* Convert $A x = b$ into a minimization problem: $x = argmin_{x} \Vert A x -b \Vert$.
* Use a Krylov space: solve for $x$ with the constraint $x \in \mathcal{K}_k$
* Use the Arnoldi algorithm to find an orthonormal basis for $\mathcal{K}_k$

## 1.2 Derivation of the Algorithm

We know the solution(s) of the normal equation satisfy $x = argmin_x \Vert A x - b \Vert.\quad$  (See [20_LengthOrthogonality.ipynb section 3.](20_LengthOrthogonality.ipynb))

Consider the Krylov Space $\quad \mathcal{K}_k(b) = \left( \; b \; A b \; A^2 b \; \dots A^{k-1} b \; \right)$ for a given choice of $k$.<br>
$\qquad$ the Arnoldi Algorithm yields a matrices $Q_m$ with orthonormal columns $q_m$, where $q_1 = \frac{1}{\Vert b \Vert} b$,<br>
$\qquad$ and a matrix $H_k$ such that $A Q_k = Q_{k+1} H_k$. (Note, $H_k$ has size $(k+1) \times k$, i.e.,<br>
$\qquad$ it includes the extra row shown in [the Arnoldi notebook](Arnoldi_Ritz_eigenvalues.ipynb).)

Choosing $x$ in the Krylov space $\mathcal{K}_k$ lets us write $x$ as a linear combination of the columns of $Q_k$, i.e., $x = Q_k \tilde{y}$ for some $\tilde{y}$

The optimization problem becomes<br>
$\qquad\begin{align}
x_k &= argmin_y \Vert\; A Q_k y - \beta q_1 \;\Vert \\
    &= argmin_y \Vert\; Q_{k+1} H_k y - \beta q_1 \;\Vert \\
    &= argmin_y \Vert\; Q_{k+1} \left( H_k y - \beta\ Q^t_{k+1} q_1 \right) \;\Vert \\
    &= argmin_y \Vert\;H_k y - \beta\ Q^t_{k+1} q_1 \;\Vert \\
    &= argmin_y \Vert\;H_k y - \beta\ e_1 \;\Vert
\end{align}$<br>

where $\beta = \Vert b \Vert$, and where we have used $\Vert Q_{k+1} z \Vert = \Vert z \Vert$ for any $z$,<br>
as well as the fact that $Q^t_{k+1} q_1 = \begin{pmatrix} 1 \\ 0 \\ \dots \\ 0 \end{pmatrix} = e_1$ since the $q_i$ are orthonormal.

___
**GMRES:** The least squares solution $x_m$ of $A x - b$ with $x \in \mathcal{K}_k$ is given by

$\qquad  y = argmin_x \Vert H_k y - \beta e_1 \Vert,\quad$ where $\beta=\Vert b \Vert$<br>
$\qquad x_k = Q_k y$

Thus $x_k$ is the $k^{th}$ approximation to the solution of $A x = b$.

# 2. Basic Implementation and Example

Given the problem $A x = b$, where $A \in \mathbb{R}^{n \times n}$.<br>
In exact arithmetic, GMRES gives the exact solution for $k=n$. The goal however is to stop well short, with $k \ll n$.

In [4]:
%%julia
function gmres(A,b,k)
    n        = length(b)
    beta     = norm(b)
    x        = 0
    residual = zeros(k)

    H      = zeros(eltype(A), k+1,k)
    Q      = zeros(eltype(A), n,k+1)
    Q[:,1] = b/beta

    for j in 1:k
        # Next step of Arnoldi iteration.
        v = A*Q[:,j]
        for i in 1:j
            H[i,j] = dot(Q[:,i],v)
            v -= H[i,j]*Q[:,i]
        end
        H[j+1,j] = norm(v)
        Q[:,j+1] = v/H[j+1,j]

        # Solve the minimum residual problem.
        r = [beta; zeros(j)]
        z = H[1:j+1,1:j] \ r
        x = Q[:,1:j]*z
        residual[j] = norm( A*x - b )
    end
    return x, residual
end;

In [5]:
%%julia
function run_gmres(N,s=1)
    A = Tridiagonal( rand(N-1), s*rand(N), rand(N-1))
    x = rand(N)
    b = A*x
    xk,residual =  gmres(A,b,N-1)
end

<PyCall.jlwrap run_gmres>

In [6]:
def show_gmres(matrix_size,main_diag_scale ):
    xk, residual = Main.run_gmres(matrix_size,main_diag_scale)
    return hv.Curve(residual, "k", "residual" )\
             .opts( title="norm(Ax - b) versus k", logy=True, show_grid=True, width=500)
interact( show_gmres, matrix_size=(50,500), main_diag_scale=(0.,20.) )

Column(sizing_mode='fixed')
    [0] Column
        [0] IntSlider(end=500, name='matrix_size', start=50, value=275)
        [1] FloatSlider(end=20.0, name='main_diag_scale', value=10.0)
    [1] Row(sizing_mode='fixed')
        [0] HoloViews(Curve, height=300, name='interactive00860', sizing_mode='fixed', width=500)